# Data Import from a Remote Db2 Warehouse on Cloud Instance

## Overview 

This notebook provides an example which connects to a Db2 Warehouse service running on IBM Cloud.  It includes the steps required to set up the data connection within IBM Cloud Private for Data to the service.  In this example, the data is stored in three different tables within Db2 Warehouse representing different business unit geographies which control the data.  The primary steps in the flow include:

- Read the data from the 3 tables in the cloud database.
- Normalize the data columns so that all 3 geographies have a common format.
- Merge the three tables into a single data set.
- Output the resulting data set to a CSV file for downstream processing.

## Create a Db2 Warehouse on Cloud Instance and Set Up Connections

A tutorial for creating a Db2 Warehouse instance on IBM Cloud is available and should be followed prior to running this notebook.  It includes steps to configure connections from this IBM Cloud Private for Data environment to the database.

## Add Data Sets

After the database has been configured and connections to it established, datasets can be added to the project.  First, from the main page of this project, select __"Data sets"__ and then __"Add Data Set"__.  

---
<img src="../misc/images/add_data_set.png" alt="Drawing" style="width: 900px;"/>
<br>

---

<br>
<br>

On the next menu, select __"Remote Data Set"__, __"Browse"__, and then your database name.  The three tables in the example database (EVENTS_EUR, EVENTS_USE, EVENTS_USW) will each need to be selected and assigned remote data set names of EUR, USE, and USW respectively.  These names will be used in the notebook's python code to read in the cooresponding table data.

<img src="../misc/images/remote_data_set.png" alt="Drawing" style="width: 400px;"/>
<br>

---

<br>

After all of the data sets have been added to your project, you will see them listed as part of the _"Data Sets"_ assets as shown below.  You can now run the rest of this notebook to read the data from the database tables, join it into a single table, and write it out to a CSV file for processing in the remaining notebooks in this industry accelerator.

---

<img src="../misc/images/final_data_sets.png" alt="Drawing" style="width: 1000px;"/>
<br>

---



**Sample Materials, provided under license. <br>
Licensed Materials - Property of IBM. <br>
© Copyright IBM Corp. 2019. All Rights Reserved. <br>
US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp.<br>**

In [8]:
import pandas as pd
import numpy as np

## Connect to the Remote Database and Read the Data

The following three cells connect the the DB2 Warehouse instance and pull the data from the three tables.  The code shown here is complete and will pull in the datasets configured as described above.  It was originally generated automatically for this notebook by following the __"Find Data"__ process as shown in the following diagram:


---
<img src="../misc/images/insert_dataframe.png" alt="Drawing" style="width: 900px;"/>
<br>

---

<br>
<br>


In [2]:
import dsx_core_utils, requests, jaydebeapi, os, io, sys
from pyspark.sql import SparkSession
import pandas as pd
df1 = None
dataSet = dsx_core_utils.get_remote_data_set_info('USW')
dataSource = dsx_core_utils.get_data_source_info(dataSet['datasource'])
if (sys.version_info >= (3, 0)):
  conn = jaydebeapi.connect(dataSource['driver_class'], dataSource['URL'], [dataSource['user'], dataSource['password']])
else:
  conn = jaydebeapi.connect(dataSource['driver_class'], [dataSource['URL'], dataSource['user'], dataSource['password']])
query = 'select * from "' + (dataSet['schema'] + '"."' if (len(dataSet['schema'].strip()) != 0) else '') +  dataSet['table'] + '"'

if (dataSet['query']):
    query = dataSet['query']
df1 = pd.read_sql(query, con=conn, parse_dates=['USW_EVENT_DATE'])
print(df1.head())

   CUSTOMER_ID USW_EVENT_DATE USW_EVENT_TYPE_ID
0         1842     2017-01-03     INT_LOGIN_WEB
1         1527     2017-01-03     INT_LOGIN_WEB
2         1068     2017-01-03     INT_LOGIN_WEB
3         1215     2017-01-03     INT_LOGIN_WEB
4         1980     2017-01-03     INT_LOGIN_WEB


In [3]:
import dsx_core_utils, requests, jaydebeapi, os, io, sys
from pyspark.sql import SparkSession
import pandas as pd
df2 = None
dataSet = dsx_core_utils.get_remote_data_set_info('USE')
dataSource = dsx_core_utils.get_data_source_info(dataSet['datasource'])
if (sys.version_info >= (3, 0)):
  conn = jaydebeapi.connect(dataSource['driver_class'], dataSource['URL'], [dataSource['user'], dataSource['password']])
else:
  conn = jaydebeapi.connect(dataSource['driver_class'], [dataSource['URL'], dataSource['user'], dataSource['password']])
query = 'select * from "' + (dataSet['schema'] + '"."' if (len(dataSet['schema'].strip()) != 0) else '') +  dataSet['table'] + '"'

if (dataSet['query']):
    query = dataSet['query']
df2 = pd.read_sql(query, con=conn, parse_dates=['USE_EVENT_DATE'])
print(df2.head())

   CUSTOMER_ID USE_EVENT_DATE     USE_EVENT_TYPE_ID
0         1900     2017-04-30  XFER_FUNDS_OUT_LARGE
1         1900     2017-04-30            XCT_EQ_BUY
2         1564     2017-04-30           XCT_EQ_SELL
3         1900     2017-04-30           XCT_EQ_SELL
4         1948     2017-04-30           XCT_EQ_SELL


In [4]:
import dsx_core_utils, requests, jaydebeapi, os, io, sys
from pyspark.sql import SparkSession
import pandas as pd
df3 = None
dataSet = dsx_core_utils.get_remote_data_set_info('EUR')
dataSource = dsx_core_utils.get_data_source_info(dataSet['datasource'])
if (sys.version_info >= (3, 0)):
  conn = jaydebeapi.connect(dataSource['driver_class'], dataSource['URL'], [dataSource['user'], dataSource['password']])
else:
  conn = jaydebeapi.connect(dataSource['driver_class'], [dataSource['URL'], dataSource['user'], dataSource['password']])
query = 'select * from "' + (dataSet['schema'] + '"."' if (len(dataSet['schema'].strip()) != 0) else '') +  dataSet['table'] + '"'

if (dataSet['query']):
    query = dataSet['query']
df3 = pd.read_sql(query, con=conn, parse_dates=['EUR_EVENT_DATE'])
print(df3.head())


   CUSTOMER_ID EUR_EVENT_DATE EUR_EVENT_TYPE_ID
0         1109     2017-01-03     INT_LOGIN_WEB
1         1166     2017-01-03     INT_LOGIN_WEB
2         1007     2017-01-03     INT_LOGIN_WEB
3         1679     2017-01-03     INT_LOGIN_WEB
4         1439     2017-01-03     INT_LOGIN_WEB


## Combine the Tables

This code sequence renames the columns of each of the tables to a uniform schema and then concatenates them into a single dataframe.


In [5]:
df1.rename({'USW_EVENT_DATE': 'EVENT_DATE', 'USW_EVENT_TYPE_ID': 'EVENT_TYPE_ID'}, axis='columns', inplace=True)
df2.rename({'USE_EVENT_DATE': 'EVENT_DATE', 'USE_EVENT_TYPE_ID': 'EVENT_TYPE_ID'}, axis='columns', inplace=True)
df3.rename({'EUR_EVENT_DATE': 'EVENT_DATE', 'EUR_EVENT_TYPE_ID': 'EVENT_TYPE_ID'}, axis='columns', inplace=True)
frames = [df1, df2, df3]
df_raw = pd.concat(frames)
print("Merged Data set:")
print(df_raw.head())

Merged Data set:
   CUSTOMER_ID EVENT_DATE  EVENT_TYPE_ID
0         1842 2017-01-03  INT_LOGIN_WEB
1         1527 2017-01-03  INT_LOGIN_WEB
2         1068 2017-01-03  INT_LOGIN_WEB
3         1215 2017-01-03  INT_LOGIN_WEB
4         1980 2017-01-03  INT_LOGIN_WEB


## Output the Final Dataset

Finally we output the dataset into a CSV file which is stored locally for processing in the subsequent steps of this application.

In [ ]:
df_raw.to_csv('../datasets/events_combined.csv', index=False)